# Modelo de recomendación de juegos

### En este apartado se realizara un modelo de recomendación, que generen una lista de 5 juegos por medio del ingreso del id de producto

- Para el modelo ingresando el id del producto tiene una relacion item-item es decir se toma un juego y en base a que tan similar al resto juegos recomienda similares.





Si es un sistema de recomendación item-item:

def recomendacion_juego( id de producto ): Ingresando el id de producto, deberíamos recibir una lista con 5 juegos recomendados similares al ingresado. Si es un sistema de recomendación user-item:

In [53]:
import pandas as pd
import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import joblib 
import pickle

In [2]:
games = pd.read_csv("./data/games.csv")

In [3]:
games

,Id_game,App_name,Release_year,Genres,Specs,Price,Early_access,Developer
0,10,Counter-Strike,2000,['Action'],"['Multi-player', 'Valve Anti-Cheat enabled']",9.99,0,Valve
1,20,Team Fortress Classic,1999,['Action'],"['Multi-player', 'Valve Anti-Cheat enabled']",4.99,0,Valve
2,30,Day of Defeat,2003,['Action'],"['Multi-player', 'Valve Anti-Cheat enabled']",4.99,0,Valve
3,40,Deathmatch Classic,2001,['Action'],"['Multi-player', 'Valve Anti-Cheat enabled']",4.99,0,Valve
4,50,Half-Life: Opposing Force,1999,['Action'],"['Single-player', 'Multi-player', 'Valve Anti-...",4.99,0,Gearbox Software
...,...,...,...,...,...,...,...,...
22503,2028055,Tom Clancy's Ghost Recon Future Soldier - Seas...,2012,['Action'],"['Single-player', 'Multi-player', 'Co-op', 'Do...",24.99,0,"Ubisoft Paris,Red Storm Entertainment"
22504,2028056,Worms Revolution Season Pass,2012,['Strategy'],"['Single-player', 'Multi-player', 'Co-op', 'Sh...",14.99,0,Team17 Digital Ltd.
22505,2028062,Call of Duty®: Black Ops II Season Pass,No_Data,['Action'],"['Single-player', 'Multi-player', 'Co-op', 'Do...",49.99,0,Treyarch
22506,2028103,Assassin’s Creed® III Season Pass,2012,"['Action', 'Adventure']","['Single-player', 'Multi-player', 'Downloadabl...",29.99,0,Ubisoft Montreal


In [4]:
df = games[['Id_game','App_name','Genres','Specs']].copy()
df.head(3)

,Id_game,App_name,Genres,Specs
0,10,Counter-Strike,['Action'],"['Multi-player', 'Valve Anti-Cheat enabled']"
1,20,Team Fortress Classic,['Action'],"['Multi-player', 'Valve Anti-Cheat enabled']"
2,30,Day of Defeat,['Action'],"['Multi-player', 'Valve Anti-Cheat enabled']"


In [5]:
# Se creara una columna nueva que contenga una categoria unica por juego en donde este el tipo de genero + la especificaciones
df['Category'] = df['Genres'] + ',' + df['Specs']

In [6]:
# Se elimina las columnas innecesarias
df = df[['Id_game','App_name','Category']]
df.columns

Index(['Id_game', 'App_name', 'Category'], dtype='object')

In [7]:
# Se pone en minuscula todas las palabras de las columnas tipo str
df['App_name'] =df['App_name'].str.lower()
df['Category'] =df['Category'].str.lower()
df.head(3)

,Id_game,App_name,Category
0,10,counter-strike,"['action'],['multi-player', 'valve anti-cheat ..."
1,20,team fortress classic,"['action'],['multi-player', 'valve anti-cheat ..."
2,30,day of defeat,"['action'],['multi-player', 'valve anti-cheat ..."


In [8]:
# Se eliminan caracteres de la columna 'Category'

def eliminar_caracteres(cadena):
    # Reemplaza los caracteres no deseados por una cadena vacía
    caracteres_a_eliminar = ["[", "]", "'", " ", '"']
    for caracter in caracteres_a_eliminar:
        cadena = cadena.replace(caracter, '')

    return cadena

df['Category'] =df['Category'].apply(eliminar_caracteres)

In [9]:
# se realiza una funcion para eliminar palabras repetidas 

def eliminar_palabras_repetidas(cadena):
    cadena = cadena.split(",")  # se divide la cadena por medio del seprador ","  quedando uan lista de palabras
    palabras_unicas = list(set(cadena))  # Se convierte la lista en conjunto y automaticamente elimina las palabras duplicadas
    texto_sin_repetidos = ', '.join(palabras_unicas) # Se convierte el conjunto a cadena uniendo las palabras por "," y un espacio
    return texto_sin_repetidos

    # Se aplica 
df['Category'] =df['Category'].apply(eliminar_palabras_repetidas)

In [65]:
df.head(3)

,Id_game,App_name,Category
0,10,counter-strike,"valveanti-cheatenabled, action, multi-player"
1,20,team fortress classic,"valveanti-cheatenabled, action, multi-player"
2,30,day of defeat,"valveanti-cheatenabled, action, multi-player"


In [66]:
# Se realiza un get dummies a la columna 'Category' en un nuevo DataFrame

df_dummies = df['Category'].str.get_dummies(', ')
df_dummies


,action,adventure,animation&amp;modeling,audioproduction,captionsavailable,casual,co-op,commentaryavailable,cross-platformmultiplayer,design&amp;illustration,...,steamleaderboards,steamtradingcards,steamturnnotifications,steamvrcollectibles,steamworkshop,strategy,utilities,valveanti-cheatenabled,videoproduction,webpublishing
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22503,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22504,0,0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,1,0,0,0,0
22505,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
22506,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [67]:
valores_unicos = df_dummies.columns.unique()
valores_unicos = sorted(valores_unicos)

In [68]:
# Se examinan los datos para depurar las columnas que no son muy necesarias
valores_unicos

['action',
 'adventure',
 'animation&amp;modeling',
 'audioproduction',
 'captionsavailable',
 'casual',
 'co-op',
 'commentaryavailable',
 'cross-platformmultiplayer',
 'design&amp;illustration',
 'downloadablecontent',
 'earlyaccess',
 'education',
 'freetoplay',
 'fullcontrollersupport',
 'gamedemo',
 'in-apppurchases',
 'includesleveleditor',
 'includessourcesdk',
 'indie',
 'localco-op',
 'localmulti-player',
 'massivelymultiplayer',
 'mmo',
 'mods',
 'mods(requirehl2)',
 'multi-player',
 'onlineco-op',
 'onlinemulti-player',
 'partialcontrollersupport',
 'photoediting',
 'racing',
 'rpg',
 'shared/splitscreen',
 'simulation',
 'single-player',
 'softwaretraining',
 'sports',
 'stats',
 'steamachievements',
 'steamcloud',
 'steamleaderboards',
 'steamtradingcards',
 'steamturnnotifications',
 'steamvrcollectibles',
 'steamworkshop',
 'strategy',
 'utilities',
 'valveanti-cheatenabled',
 'videoproduction',
 'webpublishing']

Luego de obtener una matriz y observando su tamaño se procede a convertirla en una matriz dispersa para poder almacenar solos los elementos distintos de cero junto con su ubicacion en la matriz con el fin de optimar recursos es decir ahorro de memoria, esta "conversion" se realizara por medio de la libreria SciPy haciendo uso del modulo 'scipy.sparse' que proporciona herramientas y estructuras para trabajar con matrices dispersas(sparse matrix).

In [69]:
matriz_dispersa  = sp.sparse.csr_matrix (df_dummies.values)
matriz_dispersa

<22508x51 sparse matrix of type '<class 'numpy.int64'>'
	with 151161 stored elements in Compressed Sparse Row format>

Para aplicar la similitud del coseno se hace uso del submodulo sklearn.metrics.pairwise import cosine_similarity de la biblioteca scikit-learn, este submodulo se usa para calcular la similitud del coseno entre vectores o matrices.

In [70]:
# Se crea la similitud del de coseno

item_similarity = cosine_similarity(matriz_dispersa)

In [46]:
item_similarity.shape

(22508, 22508)

In [73]:
joblib.dump(item_similarity,'modelo1.pkl')

['modelo1.pkl']

In [71]:
# Se realiza una prueba rapida para ver como se estructariria la función y como se acopla el df donde estan los 
#juegos con su respectivo id y el modelo de similitud de coseno.

def similitud_prueba(id_game, similarity_matrix, df):
    # Se encuentra el índice del juego
    game_index = df[df['Id_game'] == id_game].index[0]

    # Se calcula la similitud de coseno entre el juego seleccionado y todos los otros juegos
    similar_games_indices = similarity_matrix[game_index].argsort()[::-1][1:]

    # halla los títulos de los juegos similares a partir de los indices calculados
    similar_games_name = df['App_name'].iloc[similar_games_indices]
    
    return similar_games_name

In [72]:
# Funciona 
id_game = 10
top_similar_games_1 = similitud_prueba(id_game, item_similarity, df)
top_similar_games_1.head(5)

1            team fortress classic
2                    day of defeat
3               deathmatch classic
14    half-life deathmatch: source
5                         ricochet
Name: App_name, dtype: object

El archivo games_ML.csv es una copia de del DataFrame llamado 'df'en donde solo se dejo el numero de identificacion del juego "Id_game" y su respectivo nombre "App_name", esta seria la base de datos del juego para las busquedas.

Se hace una prueba de similitud del coseno 

In [78]:
def recomendacion_juego(id_game):
    
    end6 = pd.read_csv('games_ml.csv')
    similarity_matrix = joblib.load('modelo1.pkl')
    # Se encuentra el índice del juego
    game_index = end6[end6['Id_game'] == id_game].index[0]

    # Se calcula la similitud de coseno entre el juego seleccionado y todos los otros juegos
    similar_games_indices = similarity_matrix[game_index].argsort()[::-1][1:]

    # halla los títulos de los juegos similares a partir de los indices calculados
    similar_games_name = end6['App_name'].iloc[similar_games_indices]
    
    top5 = similar_games_name.head(5)

    return top5

In [79]:
recomendacion_juego(10)

1            Team Fortress Classic
2                    Day of Defeat
3               Deathmatch Classic
14    Half-Life Deathmatch: Source
5                         Ricochet
Name: App_name, dtype: object

#### Conclusiones

- El modelo de recomendacion funciona bien, ya que el juego proporcionado con numero de Id_game=10 es Counter Strike catalagodo como un juego de 'accion', se observa que la los resultados coinciden con la categoria de juego dada, por ejemplo el primer juego https://steamdb.info/app/20/info/, el segundo https://steamdb.info/app/30/info/, el tercero https://steamdb.info/app/40/info/, el cuarto: https://steamdb.info/app/360/info/ y el ultimo: https://steamdb.info/app/60/info/.

- Todos los juegos recomendados estan dentro de la categoria 'Action', el modelo es 100 % funcional

EL modelo que se va a montar en la API de recomendacion tiene el modelo llamado item_similarity.pkl el cual es pequeño para poder ejecutarlo en la FastApi, este cambio se hizo debido a que el modelo entrenado "item_similarity" es mas robusto en cuanto a su estructura y no es posible subir el archivo a github ni tampoco puedo ser consumido por la Api devido al limite de ram gratuito.

In [51]:
def recomendacion_juego2(id_game):

    id = int(id_game)  
    end6 = pd.read_csv('games_ml.csv')
    with open('item_similarity.pkl', 'rb') as model_file:
        # Carga el modelo serializado desde el archivo
        similarity_matrix = pickle.load(model_file)
    # Se encuentra el índice del juego
    
    game_index = end6[end6['Id_game'] == id].index[0]

    # Se calcula la similitud de coseno entre el juego seleccionado y todos los otros juegos
    similar_games_indices = similarity_matrix[game_index].argsort()[::-1][1:]

    # halla los títulos de los juegos similares a partir de los indices calculados
    similar_games_name = end6['App_name'].iloc[similar_games_indices]
    
    top5 = similar_games_name.head(5)

    return top5

In [52]:
recomendacion_juego2(10)

131                           BloodRayne 2
114      Amazing Adventures The Lost Tomb™
115        Mystery P.I.™ - The Vegas Heist
698                          Space Giraffe
116    Amazing Adventures Around the World
Name: App_name, dtype: object

-En este modelo creado para la api no es muy acertivo en cuanto ya que solo el primer juego recomendado es de accion, el resto son genero casual, esto se debe a que los datos usados en el algoritmo fueron una 1/5 parte de los datos usados en el primer modelo, por ende las recomendaciones no son muy acertivas